## CTU-13 data 전처리 과정


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
dataset=pd.read_csv('/content/drive/MyDrive/캡스톤/dataset/10-Rbot-20110818.binetflow.csv')

In [3]:
dataset

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/18 10:21:46.633335,1.060248,tcp,93.45.239.29,1611,->,147.32.84.118,6881,S_RA,0.0,0.0,4,252,132,flow=Background-TCP-Attempt
1,2011/08/18 10:19:49.027650,279.349152,tcp,62.240.166.118,1031,<?>,147.32.84.229,13363,SRPA_PA,0.0,0.0,15,1318,955,flow=Background-TCP-Attempt
2,2011/08/18 10:22:07.160628,166.390015,tcp,147.32.86.148,58067,->,66.235.132.232,80,SR_SA,0.0,0.0,3,212,134,flow=Background-TCP-Established
3,2011/08/18 10:26:02.052163,1.187083,tcp,147.32.3.51,3130,->,147.32.84.46,10010,S_RA,0.0,0.0,4,244,124,flow=Background-TCP-Attempt
4,2011/08/18 10:26:52.226748,0.980571,tcp,88.212.37.169,3134,->,147.32.84.118,6881,S_RA,0.0,0.0,4,244,124,flow=Background-TCP-Attempt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309786,2011/08/18 15:04:59.579762,0.000158,tcp,219.129.213.146,25463,->,147.32.87.44,80,S_SA,0.0,0.0,2,152,78,flow=Background-TCP-Established
1309787,2011/08/18 15:04:59.626719,0.000688,udp,213.220.104.152,64126,<->,147.32.84.229,13363,CON,0.0,0.0,2,306,244,flow=Background-UDP-Established
1309788,2011/08/18 15:04:59.686325,0.000689,udp,59.22.11.248,44432,<->,147.32.84.229,13363,CON,0.0,0.0,2,144,83,flow=Background-UDP-Established
1309789,2011/08/18 15:04:59.690632,0.000000,udp,95.102.170.66,16978,->,147.32.84.229,13363,INT,0.0,NaN,1,172,172,flow=Background-UDP-Attempt


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309791 entries, 0 to 1309790
Data columns (total 15 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   StartTime  1309791 non-null  object 
 1   Dur        1309791 non-null  float64
 2   Proto      1309791 non-null  object 
 3   SrcAddr    1309791 non-null  object 
 4   Sport      1302804 non-null  object 
 5   Dir        1309791 non-null  object 
 6   DstAddr    1309791 non-null  object 
 7   Dport      1205576 non-null  object 
 8   State      1308563 non-null  object 
 9   sTos       1296939 non-null  float64
 10  dTos       1123000 non-null  float64
 11  TotPkts    1309791 non-null  int64  
 12  TotBytes   1309791 non-null  int64  
 13  SrcBytes   1309791 non-null  int64  
 14  Label      1309791 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 149.9+ MB


In [ ]:
index=dataset[dataset['Label'].str.contains('Background')].index # 'Background' 포함하는 index 찾기
dataset= dataset.drop(index)
dataset


,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
532,2011/08/18 10:19:15.693956,3587.569824,tcp,147.32.84.164,54784,<?>,74.125.232.215,443,PA_PA,0.0,0.0,3049,978731,245317,flow=From-Normal-V51-Grill
822,2011/08/18 10:19:18.143576,198.072739,tcp,147.32.84.164,56165,<?>,74.125.232.197,80,FA_FA,0.0,0.0,14,924,462,flow=From-Normal-V51-Grill
842,2011/08/18 10:19:18.303590,197.928329,tcp,147.32.84.164,57965,<?>,209.85.149.138,80,FA_FA,0.0,0.0,14,924,462,flow=From-Normal-V51-Grill
1021,2011/08/18 10:19:19.838272,0.000399,udp,147.32.84.170,45807,<->,147.32.80.9,53,CON,0.0,0.0,2,400,74,flow=From-Normal-V51-Stribrek
1022,2011/08/18 10:19:19.839123,0.000400,udp,147.32.84.170,35380,<->,147.32.80.9,53,CON,0.0,0.0,2,400,74,flow=From-Normal-V51-Stribrek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309670,2011/08/18 15:04:58.033603,0.000743,icmp,147.32.84.164,0x0008,<->,147.32.96.69,0x841e,ECO,0.0,0.0,2,196,98,flow=From-Normal-V51-Grill
1309736,2011/08/18 15:04:59.032685,0.000913,icmp,147.32.84.164,0x0008,<->,147.32.96.69,0x841e,ECO,0.0,0.0,2,196,98,flow=From-Normal-V51-Grill
1309782,2011/08/18 15:04:59.532629,0.000414,udp,147.32.84.170,37138,<->,147.32.80.9,53,CON,0.0,0.0,2,244,81,flow=From-Normal-V51-Stribrek
1309783,2011/08/18 15:04:59.533932,0.000322,udp,147.32.84.170,39771,<->,147.32.80.9,53,CON,0.0,0.0,2,280,81,flow=From-Normal-V51-Stribrek


In [ ]:
dataset['State'].value_counts().to_csv("state3.csv")

In [ ]:
# 총 9개의 feature
# categorical 변수 - proto, dir, state, stos, dtos
# Numerical 변수 - dur, tot_pkts, tot_bytes, src_bytes
# label -> Background, Normal, Botnet
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776097 entries, 0 to 776096
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   dur        776097 non-null  float32 
 1   proto      776097 non-null  category
 2   dir        776097 non-null  category
 3   state      776031 non-null  category
 4   stos       767090 non-null  float32 
 5   dtos       722004 non-null  float32 
 6   tot_pkts   776097 non-null  int32   
 7   tot_bytes  776097 non-null  int32   
 8   src_bytes  776097 non-null  int32   
 9   label      776097 non-null  category
 10  Family     776097 non-null  category
dtypes: category(5), float32(3), int32(3)
memory usage: 22.2 MB


###Filling Null Values

In [ ]:
# Null Value 있는 feature -> state, stos, dtos
dataset.isnull().sum()

dur              0
proto            0
dir              0
state           66
stos          9007
dtos         54093
tot_pkts         0
tot_bytes        0
src_bytes        0
label            0
Family           0
dtype: int64

In [ ]:
dataset['state'].value_counts().to_csv('state.csv')


In [ ]:
# replace Null Value to Con
dataset['state'] = dataset.state.fillna(value='CON')

In [ ]:
# nan을 앞에 있는 것으로 채움
dataset['stos'] = dataset.stos.fillna(method='pad')
dataset['dtos'] = dataset.stos.fillna(method='pad')

### 필요없는 data 제거
1. 필요없는 열 -> 'Family'
2. 필요없는 행 -> Background

In [ ]:
dataset=dataset.drop(['Family'], axis=1) # 필요없는 정보라 삭제

In [ ]:
# label에서 background data 지우기
index=dataset[dataset['label'].str.contains('Background')].index # 'Background' 포함하는 index 찾기
dataset= dataset.drop(index)
dataset.reset_index(drop=True, inplace=True) # index 차례로 만들기

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21412 entries, 0 to 21411
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   dur        21412 non-null  float32 
 1   proto      21412 non-null  category
 2   dir        21412 non-null  category
 3   state      21412 non-null  category
 4   stos       21412 non-null  float32 
 5   dtos       21412 non-null  float32 
 6   tot_pkts   21412 non-null  int32   
 7   tot_bytes  21412 non-null  int32   
 8   src_bytes  21412 non-null  int32   
 9   label      21412 non-null  category
dtypes: category(4), float32(3), int32(3)
memory usage: 623.0 KB


In [ ]:
dataset['dtos'].value_counts()

0.0    21412
Name: dtos, dtype: int64

### One-hot-encdoer를 통해 이어붙이기
1. proto, stos, dtos 바꾸기 (논문제시)
2. dir 바꾸기
3. state ? -> 제거 (너무 많음, 331개)

In [ ]:
dataset['dir'].value_counts()

   ->    15177
  <->     6059
  <?>      152
   ?>       11
  who       11
  <-         2
Name: dir, dtype: int64

In [ ]:
# proto, stos, dtos, dir 모두 one-hot-encoder 로 변환
from sklearn.preprocessing import OneHotEncoder
# proto
ohe_proto = OneHotEncoder(sparse=False)
ohe_proto.fit(dataset[['proto']])
df_proto = pd.DataFrame(ohe_proto.transform(dataset[['proto']]), columns=['proto0', 'proto1', 'proto2','proto3'])

# stos
ohe_stos = OneHotEncoder(sparse=False)
ohe_stos.fit(dataset[['stos']])
df_stos = pd.DataFrame(ohe_stos.transform(dataset[['stos']]), columns=['stos0'])

# dtos
ohe_dtos = OneHotEncoder(sparse=False)
ohe_dtos.fit(dataset[['dtos']])
df_dtos = pd.DataFrame(ohe_dtos.transform(dataset[['dtos']]), columns=['dtos0'])

# dir
ohe_dir = OneHotEncoder(sparse=False)
ohe_dir.fit(dataset[['dir']])
df_dir = pd.DataFrame(ohe_dir.transform(dataset[['dir']]), columns=['dir0','dir1','dir2','dir3','dir4','dir5'])



In [ ]:
dataset

,dur,proto,dir,state,stos,dtos,tot_pkts,tot_bytes,src_bytes,label
0,3587.569824,tcp,<?>,PA_PA,0.0,0.0,3049,978731,245317,flow=From-Normal-V51-Grill
1,198.072739,tcp,<?>,FA_FA,0.0,0.0,14,924,462,flow=From-Normal-V51-Grill
2,197.928329,tcp,<?>,FA_FA,0.0,0.0,14,924,462,flow=From-Normal-V51-Grill
3,0.000399,udp,<->,CON,0.0,0.0,2,400,74,flow=From-Normal-V51-Stribrek
4,0.000400,udp,<->,CON,0.0,0.0,2,400,74,flow=From-Normal-V51-Stribrek
...,...,...,...,...,...,...,...,...,...,...
21407,3.955126,tcp,->,SPA_SPA,0.0,0.0,7,567,224,flow=From-Botnet-V51-1-TCP-Established
21408,0.000000,udp,->,INT,0.0,0.0,1,287,287,flow=From-Normal-V51-UDP-CVUT-DNS-Server
21409,0.000937,icmp,<->,ECO,0.0,0.0,2,196,98,flow=From-Normal-V51-Grill
21410,0.000913,icmp,<->,ECO,0.0,0.0,2,196,98,flow=From-Normal-V51-Grill


In [ ]:
# state 생략 후 one-hot-encdoer 변환된 부분 이어붙임
dataset = pd.concat([dataset[['dur']], df_proto, df_dir, df_stos, df_dtos,  dataset[['tot_pkts','tot_bytes','src_bytes','label']]], axis=1)

In [ ]:
dataset

,dur,proto0,proto1,proto2,proto3,dir0,dir1,dir2,dir3,dir4,dir5,stos0,dtos0,tot_pkts,tot_bytes,src_bytes,label
0,3587.569824,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3049,978731,245317,flow=From-Normal-V51-Grill
1,198.072739,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,14,924,462,flow=From-Normal-V51-Grill
2,197.928329,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,14,924,462,flow=From-Normal-V51-Grill
3,0.000399,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2,400,74,flow=From-Normal-V51-Stribrek
4,0.000400,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2,400,74,flow=From-Normal-V51-Stribrek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21407,3.955126,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,7,567,224,flow=From-Botnet-V51-1-TCP-Established
21408,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,287,287,flow=From-Normal-V51-UDP-CVUT-DNS-Server
21409,0.000937,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2,196,98,flow=From-Normal-V51-Grill
21410,0.000913,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2,196,98,flow=From-Normal-V51-Grill


1. hyperparameter 를 바꿔가면서 
2. 다른 dataset 활용
 -> network data에 대해서 


1. deep k-means
2. autoencdoer 먼저 학습하고 deep k-means

In [ ]:
normal